# Libraries

In [ ]:
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt

# Keras for TensorFlow
from tensorflow.contrib.keras.python.keras.models import Sequential
from tensorflow.contrib.keras.python.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from tensorflow.contrib.keras.python.keras.optimizers import Adam, RMSprop
from tensorflow.contrib.keras.python.keras.callbacks import EarlyStopping
from tensorflow.contrib.keras.python.keras.regularizers import l1, l2

# from tensorflow.contrib.keras.python.keras.applications.resnet50 import ResNet50

# Load training and validation sets

In [ ]:
trainingset_x = genfromtxt('gender/wiki_crop/64_64_50614_4098_training_x_onehot.csv', delimiter=',')
trainingset_y = genfromtxt('gender/wiki_crop/64_64_50614_4098_training_y_onehot.csv', delimiter=',')

testingset_x = genfromtxt('gender/wiki_crop/64_64_11938_4098_testing_x_onehot.csv', delimiter=',')
testingset_y = genfromtxt('gender/wiki_crop/64_64_11938_4098_testing_y_onehot.csv', delimiter=',')

valset_x = genfromtxt('gender/wiki_crop/64_64_5968_4098_val_x_onehot.csv', delimiter=',')
valset_y = genfromtxt('gender/wiki_crop/64_64_5968_4098_val_y_onehot.csv', delimiter=',')

In [ ]:
trainingset_x.shape

In [ ]:
trainingset_y.shape

In [ ]:
valset_x.shape

In [ ]:
valset_y.shape

# Data processing

In [ ]:
img_dim = 64
n_channels = 1
n_inputs = img_dim*img_dim
n_classes = 2

trainingset_x = trainingset_x.reshape(trainingset_x.shape[0], img_dim, img_dim)
trainingset_x = np.expand_dims(trainingset_x, axis=4)

testingset_x = testingset_x.reshape(testingset_x.shape[0], img_dim, img_dim)
testingset_x = np.expand_dims(testingset_x, axis=4)

valset_x = valset_x.reshape(valset_x.shape[0], img_dim, img_dim)
valset_x = np.expand_dims(valset_x, axis=4)

# Some useful functions

#### Get label from vector or number

In [ ]:
def get_label(ohv):
    if ohv.shape[0] == 1:
        indx = ohv[0]
    else:
        indx = np.argmax(ohv)
        
    if indx == 0:
        return 'female'
    elif indx == 1:
        return 'male'

#### Plot a sample

In [ ]:
img = trainingset_x[0, :]
img = img.reshape([img_dim, img_dim])
img = np.transpose(img)
plt.imshow(img, cmap='gray')
plt.xlabel('ground truth: ' + get_label(trainingset_y[0]))
plt.show()

In [ ]:
img = valset_x[2500, :]
# img = img.reshape([img_dim, img_dim])
img = img.reshape([img_dim, img_dim])
img = np.transpose(img)
# plt.imshow(img)
plt.imshow(img, cmap='gray')
plt.xlabel('ground truth: ' + get_label(valset_y[2500]))
plt.show()

# Define CNN model using Keras + TensorFlow

#### A model in Keras + TensorFlow defined as a sequence of layers

In [ ]:
model = Sequential()

#### The 1st conv layer followed by a max pooling layer

In [ ]:
model.add(Conv2D(64, (3, 3), activation='relu', padding='same',
                 input_shape=(img_dim, img_dim, n_channels)))

In [ ]:
model.add(Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [ ]:
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

In [ ]:
model.add(Conv2D(128, (3, 3), activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [ ]:
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

In [ ]:
model.add(Conv2D(128, (3, 3), activation='relu'))

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#### Fully connected layer

In [ ]:
n_hidden_units = 2048

model.add(Flatten())
model.add(Dense(n_hidden_units, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

# Compile the model

In [ ]:
opt = Adam(lr=0.0001, decay=10e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Training the model

In [ ]:
batch_size = 128
n_epoches = 500

# Stop training when the validation loss does not change significantly in 20 epoches
early_stopping = EarlyStopping(monitor='val_loss', patience=20)

model.fit(trainingset_x, trainingset_y,
          batch_size=batch_size, epochs=n_epoches,
          validation_data=(valset_x, valset_y),
          shuffle=True, callbacks=[early_stopping])

# Evaluating trained model

In [ ]:
scores = model.evaluate(testingset_x[0:9413 + 1, :], testingset_y[0:9413 + 1, :])
print('%s for male: %f' % (model.metrics_names[1], scores[1]))

scores = model.evaluate(testingset_x[9413:, :], testingset_y[9413:, :])
print('%s for female: %f' % (model.metrics_names[1], scores[1]))

# Save trained model

In [ ]:
model.save_weights('gender_hflip_female.h5')
print('> model saved')

# Test trained model with a sample

In [ ]:
img_indx = np.uint32(np.random.rand()*(testingset_x.shape[0] - 1))
sample = testingset_x[img_indx, :]
sample = sample.reshape([1, img_dim, img_dim, 1])

pred_cls = model.predict_classes(sample, verbose=0)

sample = sample.reshape([img_dim, img_dim])
sample = np.transpose(sample)

plt.imshow(sample, cmap='gray')
plt.show()

print('> testing image index: %d\n> true gender: %s\n> predicted gender: %s'
      % (img_indx, get_label(testingset_y[img_indx, :]), get_label(pred_cls)))

In [ ]:
import cv2

img = cv2.imread('duong.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# plt.imshow(img, cmap='gray')
plt.show()

img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
img = cv2.resize(img, (64, 64))

img = cv2.transpose(img)
img = img.reshape([1, img_dim, img_dim, 1])
pred_cls = model.predict_classes(img, verbose=0)
proba = model.predict(img, verbose=0)

print('> predicted gender: %s (%f%%)'
      % (get_label(pred_cls), np.max(proba)))
# print('> testing image index: %d\n> true gender: %s\n> predicted gender: %s'
#       % (img_indx, get_label(testingset_y[img_indx, :]), get_label(pred_cls)))